# Scraping Rotten Tomatoes

### Scraping list of movies

In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from bs4.element import NavigableString, Tag
from datetime import datetime

# This loop loops through pages of DVD & Streaming movies on Rotten Tomatoes, appending the movies in the page to a dataframe
i = 1
while True:
    # Visit page containing movies
    url = 'https://www.rottentomatoes.com/api/private/v2.0/browse?maxTomato=100&services=amazon%3Bhbo_go%3Bitunes%3Bnetflix_iw%3Bvudu%3Bamazon_prime%3Bfandango_now&certified&sortBy=release&type=dvd-streaming-all&page=' + str(i)
    response = requests.get(url)
    response_json = response.json()
    
    # If there are no results, we have reached the last page, so break out of the loop before appending nothing to the dataframe
    if response_json['results'] == []: break
    
    # If we are on the first iteration, create a new dataframe; otherwise append to it
    if i == 1:
        df = pd.DataFrame(response_json['results'])
    else:
        df = df.append(pd.DataFrame(response_json['results']))
    
    # Store dataframe so far
    df.to_pickle('rt')
    
    # Iterate counter to go to the next page in the next loop
    print(i)
    time.sleep(1)
    i += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


C:\Users\Jacob\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312


### Scraping functions

In [ ]:
### Scrape ratings
def scrape_ratings(soup):
    # Get ratings nodes
    r_nodes = soup.find_all('span', attrs = {'class': 'mop-ratings-wrap__percentage'})

    # Save ratings
    try:
        tomatometer = r_nodes[0].text.split()[0][:-1]
    except:
        tomatometer = None
    
    try:
        audience = r_nodes[1].text.split()[0][:-1]
    except:
        audience = None

    return tomatometer, audience


### Scrape release date
def scrape_release_date(soup):
    t_nodes = soup.find_all('time')
    
    try:
        release_date = t_nodes[0].text
    except:
        release_date = None
    
    return release_date


### Scrape number of ratings
def scrape_ratings_count(soup):
    tc_nodes = soup.find_all('small', attrs = {'class': 'mop-ratings-wrap__text--small'})
    
    try:
        total_count = tc_nodes[0].text.split()[0]
    except:
        total_count = None
    
    ac_nodes = soup.find_all('strong', attrs = {'class': 'mop-ratings-wrap__text--small'})
    
    try:
        audience_count = ac_nodes[1].text.split()[2]
    except:
        audience_count = None
    
    return total_count, audience_count


### Scrape synopsis
def scrape_synopsis(soup):
    ms_nodes = soup.find_all('div', attrs = {'id': 'movieSynopsis'})
    
    try:
        movie_synopsis = ms_nodes[0].text.split('\n')[1].lstrip()
    except:
        movie_synopsis = None
    
    return movie_synopsis


### Scrape genres
def scrape_genres(soup):
    g_nodes = soup.find_all('div', attrs = {'class': 'meta-value'})

    genres = []
    for item in g_nodes:
        for child in item.children:
            if isinstance(child, NavigableString):
                continue
            if isinstance(child, Tag):
                try:
                    if 'genres' in child['href']:
                        genres.append(child.text)
                except:
                    continue

    return genres


### Scrape directors
def scrape_directors(soup):
    d_nodes = soup.find_all('li', attrs = {'class': 'meta-row clearfix'})

    directors = []
    next_one = False
    for item in d_nodes:
        if next_one == True: break
        for child in item.children:
            if isinstance(child, NavigableString):
                continue
            if isinstance(child, Tag):
                if next_one == True:
                    for lchild in child.children:
                        if isinstance(lchild, NavigableString):
                            continue
                        if isinstance(lchild, Tag):
                            try:
                                directors.append(lchild.text)
                            except:
                                continue
                if 'Directed By:' in child.text:
                    next_one = True

    return directors


### Scrape studios
def scrape_studios(soup):
    s_nodes = soup.find_all('a', attrs = {'target': 'movie-studio'})
    try:
        movie_studio = s_nodes[0].text
    except:
        movie_studio = None
    return movie_studio

### Loop to iterate through movies in dataframe

In [ ]:
ratings = []
release_dates = []
number_of_ratings = []
synopses = []
genres = []
directors = []
studios = []
for i in df['url']:
    url = 'https://www.rottentomatoes.com' + i
    response = requests.get(url)
    html = response.text
    soup = bs(html,'lxml')
    
    print(i)          
    
    try:
        ratings.append(scrape_ratings(soup))
    except:
        ratings.append(None)
    
    try:
        release_dates.append(scrape_release_date(soup))
    except:
        release_dates.append(None)
    
    try:
        number_of_ratings.append(scrape_ratings_count(soup))
    except:
        number_of_rating.append(None)
        
    try:
        synopses.append(scrape_synopsis(soup))
    except:
        synopses.append(None)
        
    try:
        genres.append(scrape_genres(soup))
    except:
        genres.append(None)
        
    try:
        directors.append(scrape_directors(soup))
    except:
        directors.append(None)
    
    try:
        studios.append(scrape_studios(soup))
    except:
        studios.append(None)
        
df['ratings'] = ratings
df['releaseDate'] = release_dates
df['numberOfRatings'] = number_of_ratings
df['synopsis'] = synopses
df['genres'] = genres
df['directors'] = directors
df['studio'] = studios

### Structuring data

In [ ]:
def get_minutes(string):
    if 'hr.' in string and 'min.' in string:
        hours = int(string[0:2].strip())
        minutes = int(string[-7:-5].strip())
    elif 'min.' in string:
        hours = 0
        minutes = int(string[-7:-5].strip())
    elif 'hr.' in string:
        hours = int(string[0:2].strip())
        minutes = 0
    else:
        return None
    return 60 * hours + minutes

def get_datetime(string):
    try:
        output = datetime.strptime(string, '%b %d, %Y')
    except:
        output = None
    return output

def get_nth_ele(lst, n):
    return lst[n]

# Drop useless columns
df = df.drop(['id', 'theaterReleaseDate', 'dvdReleaseDate', 'mainTrailer', 'posters', 'synopsisType', 'tomatoScore'], axis = 1)

# Drop NaN's for the first time
df = df.dropna()

# Get runtime in minutes
df['runtime'] = df['runtime'].apply(get_minutes)

# Get release_date in datetime
df['releaseDate'] = df['releaseDate'].apply(get_datetime)

# Create seperate columns
df['tomatoMeter'] = df['ratings'].apply(get_nth_ele, n = 0)
df['audienceScore'] = df['ratings'].apply(get_nth_ele, n = 1)
df['tomatoCount'] = df['numberOfRatings'].apply(get_nth_ele, n = 0)
df['audienceCount'] = df['numberOfRatings'].apply(get_nth_ele, n = 1)

# Drop columns for the second time
df = df.drop(['ratings', 'numberOfRatings'], axis = 1)

# Drop NaN's for the second time
df = df.dropna()

df